# Obtenção dos dados

#### Separamos esse trecho do código para deixar mais organizado, além de não precisar fazer a extração dos dados diretamente do MYSQL em cada execução do código principal.

Escolhemos trabalhar com a base de dados criada para a A1, que contém dados nutricionais de lanches de três redes de comida distintas: Mc Donalds, Burguer King e Starbucks. O primeiro passo, então, foi carregar os dados que estavam no MySQL para o Python utilizando as bibliotecas Pandas e Pymysql.

In [1]:
import pandas as pd
import pymysql
import getpass

Criação da conexão com o banco local:

In [2]:
p = getpass.getpass()
conexao = pymysql.connect(
    host='localhost',
    user='root',
    port=3306,
    passwd=p,
    db="fast_food")

········


Queremos fazer uma visualização que permita comparar os valores nutricionais dos lanches (através do tamanho das arestas), para isso importamos todos os lanches, sua tabela nutricional, a rede a que pertence e sua categoria. Esses últimos serão úteis para filtrar os dados.

In [3]:
query = """
SELECT p.name, n.*, c.name as "categoria", p.chain_id_chain as "rede"
FROM nutricional_units n
INNER JOIN product p ON n.id_nutricional_units = p.nutricional_units_id_nutricional_units
INNER JOIN category c ON p.category_id_category = c.id_category;
"""

df = pd.read_sql(query, con=conexao, index_col = "name")

# Tratamento dos dados

#### Para que a visualização do grafo ficasse da maneira que planejamos, precisamos tratar os dados obtidos.

A primeira coluna do df é a id do nutriente, que não será necessária, então começamos a excluindo:

In [4]:
df = df.drop(["id_nutricional_units"],axis=1)

Queremos que o tamanho (grossura) da aresta represente a quantidade presente do nutriente no lanche conectado. Como o range dentre os diferentes nutrientes é muito diferente, precisamos normalizar os dados para melhor visualização:

In [5]:
col=[x for x in df.columns[0:len(df.columns)-2]] #pegar os nomes das colunas de nutrientes (-2 exclui "categoria" e "rede")
nutrientes = df[col].copy() # criar uma cópia, somente com as colunas dos nutrientes

nutri_norm = (nutrientes - nutrientes.mean()) / nutrientes.std() #normalizar os nutrientes
nutri_min = abs(nutri_norm.min())

nutri_norm = nutri_norm + nutri_min + 1 #deixar todos os valores > 0

for nutriente in col:
    df[nutriente] = nutri_norm[nutriente] # aplica a normalização feita no df original

Iremos representar as redes pelas suas cores características, para facilitar o processo, trocamos cada nome no dataframe pela cor que queremos aplicar:

In [6]:
#atribuir cor de acordo com as redes
cores = {1:"#0055AA",2:"#CF1C24",3:"#24704D"}  

df.replace({'rede': cores},inplace=True)

Os NA podem ser trocados por 0 no nosso problema, pois de qualquer modo não serão representados pelos nodes. Caso fossem ignorados todos os dados que possuem NA, perderíamos muitas informações, pois muitos lanches seriam excluidos da base.

In [7]:
df.fillna(0,inplace=True)

Por fim, simplismente renomeamos as colunas dos nutrientes, pois serão os nomes utilizados nos nodes e no banco estão em inglês, além de alguns contem "_", o que não seria agradável na visualização do grafo.

In [8]:
# 'col' foi definido acima e seleciona somente as colunas dos nutrientes
col_names = ['calorias','gorduras totais','gorduras saturadas','gorduras trans','colesterol','sódio','carboidratos','fibras','açúcares','proteínas','vitamina A','vitamina C','cálcio','ferro', 'sal', 'cafeína']

df = df.rename(columns=dict(zip(col,col_names))) #renomeia as colunas dos nutrientes com os novos nomes (na lista col_names)

Por último, renomeamos as categorias para usarmos, posteriormente, como filtros. Nesse novo formato, temos menos categorias, o que tornará o processo futuro mais simples e organizado.

In [9]:
cat = df.categoria.unique().copy() #pegar as categorias antigas

# definir novas categorias (poderia ser feito com regex também)
novas = ['SANDUICHE DE CARNE', 'SANDUICHE DE CARNE', 'SANDUICHE DE FRANGO/PEIXE','VEGGIES','ACOMPANHAMENTO','MOLHO','SOBREMESA','SMOOTHIE/SHAKE','CAFE DA MANHA', 'SANDUICHE DE CARNE','SANDUICHE DE FRANGO/PEIXE','SALADA','ACOMPANHAMENTO','SOBREMESA','BEBIDAS','BEBIDAS','SMOOTHIE/SHAKE',
 'CAFE DA MANHA','VEGGIES','VEGGIES', 'SOBREMESA','SOBREMESA','VEGGIES','CAFE DA MANHA','CAFE DA MANHA','OUTROS SANDUICHES','OUTROS SANDUICHES','OUTROS SANDUICHES','SALADA','BEBIDAS',
 'BEBIDAS', 'BEBIDAS','BEBIDAS','BEBIDAS','BEBIDAS','BEBIDAS','BEBIDAS','BEBIDAS','BEBIDAS','SMOOTHIE/SHAKE','BEBIDAS','BEBIDAS']


df.replace(dict(zip(cat,novas)),inplace=True) #atribuir as novas categorias
df.categoria.unique() #imprimir as novas categorias

array(['SANDUICHE DE CARNE', 'SANDUICHE DE FRANGO/PEIXE', 'VEGGIES',
       'ACOMPANHAMENTO', 'MOLHO', 'SOBREMESA', 'SMOOTHIE/SHAKE',
       'CAFE DA MANHA', 'SALADA', 'BEBIDAS', 'OUTROS SANDUICHES'],
      dtype=object)

# Resultado

#### Após esses passos iniciais obtemos o seguinte data frame:

In [10]:
df.head(3)

,calorias,gorduras totais,gorduras saturadas,gorduras trans,colesterol,sódio,carboidratos,fibras,açúcares,proteínas,vitamina A,vitamina C,cálcio,ferro,sal,cafeína,categoria,rede
name,,,,,,,,,,,,,,,,,,
WHOPPER,3.505575,2.984077,2.897831,1.0,0.0,1.002474,1.214283,1.371581,0.0,3.481996,0.0,0.0,0.0,0.0,0.0,0.0,SANDUICHE DE CARNE,#0055AA
WHOPPER DUPLO,4.612173,4.287898,3.696918,1.0,0.0,1.002474,1.198410,1.557372,0.0,5.226101,0.0,0.0,0.0,0.0,0.0,0.0,SANDUICHE DE CARNE,#0055AA
WHOPPER FURIOSO,4.101435,3.380892,2.797945,1.0,0.0,1.004948,1.234124,1.371581,0.0,3.616158,0.0,0.0,0.0,0.0,0.0,0.0,SANDUICHE DE CARNE,#0055AA


# Exportação do data frame

In [11]:
df.to_csv("fastfood.csv")